In [1]:
from collections import defaultdict
from os.path import join, exists, splitext, basename
from os import listdir, symlink, makedirs
from shutil import copyfile
from praatio import tgio
from tqdm import tqdm
from glob import glob
import librosa
import pandas as pd

In [2]:
# directory where the data resides
data_root = '/data/coswara-15-03-21/'

In [3]:
# src and destination directories
load_dir = join(data_root, 'raw')
save_root = join(data_root, 'processed')

In [4]:
makedirs(save_root, exist_ok=True)

In [5]:
load_audio_dir = join(load_dir, 'audio')
load_annotation_dir = join(load_dir, 'annotations')

In [14]:
save_audio_dir = join(save_root, 'audio')
makedirs(save_audio_dir, exist_ok=True)

### Create symlinks to the original .wav files

In [8]:
date_dirs = glob(join(load_audio_dir, '*'))

user_dirs = []
for date in date_dirs:
    user_dirs.append(glob(join(date, '*')))

In [16]:
!ls /data/coswara-15-03-21

processed


In [15]:
!ls $load_audio_dir

ls: cannot access '/data/coswara-15-03-21/raw/audio': No such file or directory


In [9]:
date_dirs

[]

In [ ]:
for user_dir in tqdm(user_dirs):
    user_name = basename(user_dir)
    date = basename(basename(user_dir))

    user_audio_files = glob(join(user_dir, '*.wav'))
    for audio_file in user_audio_files:
        file_name = basename(audio_file)
        dest = join(save_audio_dir, '{}-{}-{}'.format(date, user_name, file_name))
        if not exists(dest):
            symlink(audio_file, dest)
    break

### Create attributes file

### Create annotations file